# Gravity column

This PRST example is based on the "1ph/gravityColumn.m" example found in MRST.

## MATLAB code

The code we are trying to convert to Python is the following:

```
%% My First Flow Solver: Gravity Column
% In this example, we introduce a simple pressure solver and use it to
% solve the single-phase pressure equation
%
% $$\nabla\cdot v = q, \qquad
%    v=\textbf{--}\frac{K}{\mu} \bigl[\nabla p+\rho g\nabla z\bigr],$$
%
% within the domain [0,1]x[0,1]x[0,30] using a Cartesian grid with
% homogeneous isotropic permeability of 100 mD. The fluid has density 1000
% kg/m^3 and viscosity 1 cP and the pressure is 100 bar at the top of the
% structure.
%
% The purpose of the example is to show the basic steps for setting up,
% solving, and visualizing a flow problem. More details on the grid
% structure, the structure used to hold the solutions, and so on, are given
% in the <simpleBC.html basic flow-solver tutorial>.
try
    require incomp
catch %#ok<CTCH>
    mrstModule add incomp
end

%% Define the model
% To set up a model, we need: a grid, rock properties (permeability), a
% fluid object with density and viscosity, and boundary conditions.
gravity reset on
G          = cartGrid([1, 1, 30], [1, 1, 30]);
G          = computeGeometry(G);
rock.perm  = repmat(0.1*darcy(), [G.cells.num, 1]);
fluid      = initSingleFluid('mu' ,    1*centi*poise, ...
                             'rho', 1014*kilogram/meter^3);
bc  = pside([], G, 'TOP', 100.*barsa());

%% Assemble and solve the linear system
% To solve the flow problem, we use the standard two-point
% flux-approximation method (TPFA), which for a Cartesian grid is the same
% as a classical seven-point finite-difference scheme for Poisson's
% equation. This is done in two steps: first we compute the
% transmissibilities and then we assemble and solve the corresponding
% discrete system.
T   = computeTrans(G, rock);
sol = incompTPFA(initResSol(G, 0.0), G, T, fluid, 'bc', bc);

%% Plot the face pressures
clf
plotFaces(G, 1:G.faces.num, convertTo(sol.facePressure, barsa()));
set(gca, 'ZDir', 'reverse'), title('Pressure [bar]')
view(3), colorbar
set(gca,'DataAspect',[1 1 10]);
%%
displayEndOfDemoMessage(mfilename)
```

The rest of the example will walk through this example line for line, displaying the PRST examples.

## Setup

**Start by importing PRST.**

In [20]:
# For Python 3 compatibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np

import prst
import prst.incomp as incomp
import prst.gridprocessing as gridprocessing
import prst.utils as utils
import prst.params as params
import prst.solvers as solvers

from prst.utils.units import *

In [21]:
help(prst)

Help on package prst:

NAME
    prst - Python Reservoir Simulation Toolbox.

FILE
    m:\prst\prst\__init__.py

PACKAGE CONTENTS
    gridprocessing
    incomp (package)
    io
    libgeometry (package)
    params (package)
    plotting
    solvers
    utils (package)

DATA
    __all__ = ['gridprocessing', 'io', 'incomp', 'plotting', 'utils', 'par...




## Import the "incomp" module
```
try
    require incomp
catch %#ok<CTCH>
    mrstModule add incomp
end
```

```
%% Define the model
% To set up a model, we need: a grid, rock properties (permeability), a
% fluid object with density and viscosity, and boundary conditions.
gravity reset on
```

In [22]:
# PRST uses a single module-level variable to set the gravity
prst.gravity_reset()
print("Gravity vector:", prst.gravity)

Gravity vector: [ 0.       0.       9.80665]


## Build the grid, setup fluid

```G          = cartGrid([1, 1, 30], [1, 1, 30]);
G          = computeGeometry(G);
rock.perm  = repmat(0.1*darcy(), [G.cells.num, 1]);
fluid      = initSingleFluid('mu' ,    1*centi*poise, ...
                             'rho', 1014*kilogram/meter^3);
                             ```

In [23]:
G = gridprocessing.cartGrid([1, 1, 30], [1, 1, 30])
print(G)

<PRST grid
  gridType: ['tensorGrid', 'cartGrid']
  cells: ['facePos', 'num', 'indexMap', 'faces']
  cartDims: [ 1  1 30]
  gridDim: 3
  faces: ['neighbors', 'nodes', 'num', 'tag', 'nodePos']
  nodes: ['num', 'coords']
>


**Computing the geometry modifies the original grid in place, no assignment is 
necessary. Note that specifying the module is necessary in PRST.
This avoids polluting the global namespace, at the cost of more verbosity.**

In [24]:
gridprocessing.computeGeometry(G)

INFO:prst:Computing normals, areas and centroids...
INFO:prst:Computing cell volumes and centroids


In [25]:
#import scipy
#from scipy.sparse import csr_matrix

In [26]:
rock = params.rock.Rock(G, perm=0.1*darcy, poro=1)

In [27]:
fluid = incomp.fluid.SingleFluid(viscosity=1*centi*poise, density=1014*kilogram/meter**3)
print(fluid)

{'mu': 0.001, 'rho': 1014.0}


**Set up boundary conditions. Constant pressure at top face.**

In [28]:
bc = params.wells_and_bc.BoundaryCondition()
bc.addPressureSide(G, "top", 100*bar)



In [29]:
print(bc)

{'sat': None, 'type': array([['pressure']], 
      dtype='|S8'), 'value': array([[ 10000000.]]), 'face': array([[120]])}


## Assemble and solve the linear system
```
%% Assemble and solve the linear system
% To solve the flow problem, we use the standard two-point
% flux-approximation method (TPFA), which for a Cartesian grid is the same
% as a classical seven-point finite-difference scheme for Poisson's
% equation. This is done in two steps: first we compute the
% transmissibilities and then we assemble and solve the corresponding
% discrete system.
T   = computeTrans(G, rock);
sol = incompTPFA(initResSol(G, 0.0), G, T, fluid, 'bc', bc);
```

In [30]:
T = solvers.computeTrans(G, rock)
resSol = solvers.initResSol(G, p0=0.0)

In [31]:
# prst/solvers.py

def capPressureRHS():
    pass

def computePressureRHS():
    pass

def incompTPFA():
    pass

def _dynamic_quantities():
    pass

def _compute_trans():
    pass

## getCellNoFaces

In [32]:

# prst/gridtools.py
# Find the cell index of each face for each cell

def getCellNoFaces(G):
    """
    %Get a list over all half faces, accounting for possible NNC
    %
    % SYNOPSIS:
    %   [cellNo, cellFaces, isNNC] = getCellNoFaces(G)
    %
    % DESCRIPTION:
    %   This utility function is used to produce a listing of all half faces
    %   in a grid along with the respective cells they belong to. While
    %   relatively trivial for most grids, this function specifically accounts
    %   for non-neighboring connections / NNC.
    %
    % REQUIRED PARAMETERS:
    %   G         - Grid structure with optional .nnc.cells field.
    %
    % RETURNS:
    %   cellNo    - A list of length number of geometric half-faces + 2* no. nnc
    %             connections where each entry corresponds to the cell index of
    %             that half face.
    %
    %   cellFaces - A list of length number of geometric half-faces + 2* number
    %             of nnc connections where each entry is the connection index.
    %             For the first entries, this is simply the face number.
    %             Otherwise, it is the entry of the NNC connection.
    %
    %   isNNC     - A list with the same length as cellNo / cellFaces,
    %               containing a boolean indicating if that specific connection
    %               originates from a geometric face or a NNC connection
    % SEE ALSO:
    %   rldecode
    """
    cellNo = utils.rldecode(np.arange(G.cells.num), np.diff(G.cells.facePos,axis=0))[:,np.newaxis]
    cellFaces = G.cells.faces[:,0][:,np.newaxis]

    # Mapping to show which entries in cellNo/cellFaces are resulting from
    # NNC and not actual geometric faces.
    isNNC = np.zeros((len(cellNo), 1), dtype=np.bool)

    # If NNC is present, we add these extra connections to cellNo 
    # and cellFaces to allow consisten treatment.
    if hasattr(G, 'nnc') and hasattr(G.nnc, "cells"):
        prst.warning("getCellNoFaces is untested for grids with NNC. Compare results with MRST.")
        # Stack columns into a single column
        nnc_cells = np.r_[G.nnc.cells[:,0], G.nnc.cells[:,1]][:,np.newaxis]
        # NNC is located at the end after the regular faces
        nnc_faceno = G.faces.num + np.arange(G.nnc.cells.shape[0])[:,np.newaxis]
        cellNo = np.r_[cellNo, nnc_cells] # Stack as 2d column
        cellFaces = np.r_[cellFaces, nnc_faceno, nnc_faceno] # Stack as column
        # Added connections are NNC
        isNNC = np.r_[isNNC, np.ones((len(nnc_cells),1), dtype=np.bool)]
    return cellNo, cellFaces, isNNC

## computePressureRHS

In [64]:
from numpy_groupies.aggregate_numpy import aggregate

def computePressureRHS(G, omega, bc, src):
    """
    %Compute right-hand side contributions to pressure linear system.
    %
    % SYNOPSIS:
    %   [f, g, h, grav, dF, dC] = computePressureRHS(G, omega, bc, src)
    %
    % DESCRIPTION:
    %   The contributions to the right-hand side for mimetic, two-point and
    %   multi-point discretisations of the equations for pressure and total
    %   velocity
    %
    %     v + lam KÂ·grad (p - gÂ·z omega) = 0
    %     div v  = q
    %
    %   with
    %             __
    %             \    kr_i/
    %     lam   = /       / mu_i
    %             -- i
    %             __
    %             \
    %     omega = /    f_iÂ·rho_i
    %             -- i
    %
    % PARAMETERS:
    %   G     - Grid data structure.
    %
    %   omega - Accumulated phase densities \rho_i weighted by fractional flow
    %           functions f_i -- i.e., omega = \sum_i \rho_i f_i.  One scalar
    %           value for each cell in the discretised reservoir model, G.
    %
    %   bc    - Boundary condition structure as defined by function 'addBC'.
    %           This structure accounts for all external boundary conditions to
    %           the reservoir flow.  May be empty (i.e., bc = struct([])) which
    %           is interpreted as all external no-flow (homogeneous Neumann)
    %           conditions.
    %
    %   src   - Explicit source contributions as defined by function
    %           'addSource'.  May be empty (i.e., src = struct([])) which is
    %           interpreted as a reservoir model without explicit sources.
    %
    % RETURNS:
    %   f, g, h - Pressure (f), source/sink (g), and flux (h) external
    %             conditions.  In a problem without effects of gravity, these
    %             values may be passed directly on to linear system solvers
    %             such as 'schurComplementSymm'.
    %
    %   grav    - Pressure contributions from gravity,
    %
    %                grav = omegaÂ·gÂ·(x_face - x_cell)
    %
    %             where
    %
    %                omega = \sum_i f_i\rho_i,
    %
    %             thus grav is a vector with one scalar value for each
    %             half-face in the model (size(G.cells.faces,1)).
    %
    %   dF, dC  - Dirichlet/pressure condition structure.  Logical array 'dF'
    %             is true for those faces that have prescribed pressures, while
    %             the corresponding prescribed pressure values are listed in
    %             'dC'.  The number of elements in 'dC' is SUM(DOUBLE(dF)).
    %
    %             This structure may be used to eliminate known face pressures
    %             from the linear system before calling a system solver (e.g.,
    %             'schurComplementSymm').
    %
    % SEE ALSO:
    %   addBC, addSource, computeMimeticIP, schurComplementSymm.
    """
    if hasattr(G, "grav_pressure"):
        gp = G.grav_pressure(G, omega)
    else:
        gp = _grav_pressure(G, omega)
        
    ff = np.zeros(gp.shape)
    gg = np.zeros((G.cells.num, 1))
    hh = np.zeros((G.faces.num, 1))
    
    # Source terms
    if not src is None:
        prst.warning("computePressureRHS is untested for src != None")
        # Compatability check of cell numbers for source terms
        assert np.max(src.cell) < G.cells.num and np.min(src.cell) >= 0, \
            "Source terms refer to cell not existant in grid."
        
        # Sum source terms inside each cell and add to rhs
        ss = aggregate(src.cell, src.rate)
        ii = aggregate(src.cell, 1) > 0
        gg[ii] += ss[ii]
    
    dF = np.zeros((G.faces.num, 1), dtype=bool)
    dC = None
    
    if not bc is None:
        # Check that bc and G are compatible
        assert np.max(bc.face) < G.faces.num and np.min(bc.face) >= 0, \
            "Boundary condition refers to face not existant in grid."
        assert np.all(aggregate(bc.face, 1)) <= 1, \
            "There are repeated faces in boundary condition."
            
        # Pressure (Dirichlet) boundary conditions.
        # 1) Extract the faces marked as defining pressure conditions.
        #    Define a local numbering (map) of the face indices to the
        #    pressure condition values.
        is_press = bc.type == "pressure"
        face = bc.face[is_press]
        dC = bc.value[is_press]
        map = scipy.sparse.csc_matrix( (np.arange(face.size),
                                     (face.ravel(), np.zeros(face.size)))  )
        
        # 2) For purpose of (mimetic) pressure solvers, mark the "face"s as
        #    having pressure boundary conditions. This information will be used
        #    to eliminate known pressures from the resulting system of linear
        #    equations. See e.g. `solveIncompFlow` in MRST.
        dF[face] = True
        
        # 3) Enter Dirichlet conditions into system right hand side.
        #    Relies implicitly on boundary faces being mentioned exactly once
        #    in G.cells.faces[:,0].
        i = dF[G.cells.faces[:,0],:]
        ff[i] = -dC[map[ G.cells.faces[i[:,0],0],0].toarray().ravel()]
        
        # 4) Reorder Dirichlet conditions according to sort(face).
        #    This allows the caller to issue statements such as 
        #    `X[dF] = dC` even when dF is boolean.
        dC = dC[map[dF[:,0],0].toarray().ravel()]
        
        # Flux (Neumann) boundary conditions.
        # Note negative sign due to bc.value representing INJECTION flux.
        is_flux = bc.type == "flux"
        hh[bc.face[is_flux],0] = -bc.value[is_flux]
        
    if not dC is None:
        assert not np.any(dC < 0)
        
    return ff, gg, hh, gp, dF, dC
        
def _grav_pressure(G, omega):
    # Computes innerproduct cf (face_centroid - cell_centroid) * g for each face
    g_vec = prst.gravity
    if np.linalg.norm(g_vec[:G.gridDim]) > 0:
        dim = G.gridDim
        assert 1 <= dim <= 3, "Wrong grid dimension"
        
        cellno = utils.rldecode(np.arange(G.cells.num), np.diff(G.cells.facePos, axis=0))
        cvec = G.faces.centroids[G.cells.faces[:,0],:] - G.cells.centroids[cellno,:]
        ff = omega[cellno] * np.dot(cvec, g_vec[:dim].reshape([3,1]))
    else:
        ff = np.zeros([G.cells.faces.shape[0], 1])
    return ff

In [34]:
np.reshape([1,2,3], [3,-1])

array([[1],
       [2],
       [3]])

## incompTPFA

In [51]:
# prst/solvers.py
def _compute_trans(G, T, cellNo, cellFaces, neighborship, totmob, use_trans):
    niface = neighborship.shape[0]
    if use_trans:
        neighborcount = np.sum(neighborship != -1, axis=1, keepdims=True)
        assert T.shape[0] == niface, \
            "Expected one transmissibility for each interface " + \
            "(={}) but got {}".format(niface, T.shape[0])
        raise NotImplementedError("Function not yet implemented for use_trans=True. See source code.")
        # Matlab code for rest of function, from mrst-2015b\modules\incomp\incompTPFA.m
        #fmob = accumarray(cellFaces, totmob(cellNo), ...
        #                    [niface, 1]);
        #
        #fmob = fmob ./ neighborcount;
        #ft   = T .* fmob;
        #
        #% Synthetic one-sided transmissibilities.
        #th = ft .* neighborcount;
        #T  = th(cellFaces(:,1));
    else:
        # Define face transmissibility as harmonic average of mobility 
        # weighted one-sided transmissibilities.
        assert T.shape[0] == cellNo.shape[0], \
            "Expected one one-sided transmissibility for each " +\
            "half face (={}), but got {}.".format(cellNo.shape[0], T.shape[0])

        T = T * totmob[cellNo[:,0],:]
        from numpy_groupies.aggregate_numpy import aggregate
        ft = 1/aggregate(cellFaces[:,0], 1/T[:,0])
    return T, ft

In [36]:
def _dynamic_quantities(state, fluid):
    mu, rho = fluid.viscosity(), fluid.density()
    s = fluid.saturation(state)
    kr = fluid.relperm(s)

    mob = kr / mu
    totmob = np.sum(mob, axis=1, keepdims=True)
    omega = np.sum(mob*rho, axis=1, keepdims=True) / totmob
    return mob, omega, rho

In [74]:
import time
import prst
import scipy.sparse.linalg



state = resSol
G = G
T = T
fluid = fluid
wells = None
bc = bc
bcp = None
src = None
LinSolve = None
MatrixOutput = False
verbose = True
condition_number = False
gravity = None
pc_form = "nonwetting"
use_trans = False

if wells is None:
    wells = []
if LinSolve is None:
    LinSolve = scipy.sparse.linalg.spsolve
if verbose is None:
    verbose = prst.verbosity
if gravity is None:
    gravity = prst.gravity

    
g_vec = gravity
# If gravity is overridden, we cannot say anything about the effects of gravity on rhs.
grav = np.linalg.norm(g_vec[0:G.gridDim]) > 0 or hasattr(G, 'grav_pressure')

if all([not MatrixOutput, 
        bc is None,
        src is None,
        bcp is None,
        wells is None,
        not grav]):
    prst.log.warn("No external driving forces present in model, state remains unchanged.")
    
if verbose:
    print("Setting up linear system...")
    t0 = time.time()

# Preliminaries
neighborship, n_isnnc = utils.gridtools.getNeighborship(G, "Topological", True, nargout=2)
utils.gridtools.getCellNoFaces = getCellNoFaces # TODO 
cellNo, cf, cn_isnnc = getCellNoFaces(G) # TODO
nif = neighborship.shape[0]
ncf = cf.shape[0]
nc = G.cells.num
nw = len(wells)
n = nc + nw

mob, omega, rho = _dynamic_quantities(state, fluid)
totmob = np.sum(mob, axis=1, keepdims=True)

# Compute effective (mobility-weighted) transmissibilities
T, ft = _compute_trans(G, T, cellNo, cf, neighborship, totmob, use_trans=False)

# Identify internal faces
i = np.all(neighborship != -1, axis=1)

# Boundary conditions and source terms
hh = np.zeros((nif, 1))
dF = np.zeros((nif, 1), dtype=bool)
grav, ff = np.zeros((ncf, 1)), np.zeros((ncf, 1))
cn_notnnc = np.logical_not(cn_isnnc)[:,0]
raise NotImplementedError("complete the function")
# ff, gg, grav, dF, dC
#ff[cn_notnnc,:], gg, hh[cn_notnnc,:], grav[cn_notnnc,:], dF[cn_notnnc,:], dC = computePressureRHS(G, omega, bc, src)

Setting up linear system...


NotImplementedError: complete the function

In [73]:
LOL[5]

array([ 10000000.])

In [ ]:
fff = np.array([[121, 122, 123]]).transpose()
